In [1]:
!pip install pytorch-lightning
!pip install torch-geometric
!pip install energyflow
!pip install tensorcircuit
!pip install jax
!pip install jaxlib
!pip install torch-cluster

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 44.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [2]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 68.5 MB/s eta 0:00:00


In [3]:
!pip install torch-scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=3618829 sha256=ee93fb1524b3aaa549ce1b9e1f4bc15eb1e28cca8a3f2689c6692b2c0f5294c1
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch-scatter


In [4]:
import os
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import knn_graph, global_mean_pool
from sklearn.model_selection import train_test_split
import energyflow as ef
import pennylane as qml
import numpy as np

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
print("Loading dataset...")
qg_dataset = ef.qg_jets.load(num_data=5000, pad=True, ncol=4, generator='pythia')
x = qg_dataset[0]  # Jet features (pT, y, phi, energy)
y = torch.tensor(qg_dataset[1], dtype=torch.long)

Loading dataset...


In [7]:
def construct_graph(jet_features, label, k=5):
    """Converts a jet (list of particles) into a graph using k-NN connectivity."""
    node_features = torch.tensor(jet_features, dtype=torch.float)
    edge_index = knn_graph(node_features, k=k, loop=False)

    return Data(x=node_features, edge_index=edge_index, y=torch.tensor([label], dtype=torch.long))

jet_graphs = [construct_graph(x[i], y[i], k=5) for i in range(len(x))]

In [ ]:
train_data, val_data = train_test_split(jet_graphs, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(jet_graphs, test_size=0.1, random_state=42)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)

In [ ]:
def default_EDU(bond_params, rzz_param, wires):
    """EDU (Entanglement Driven Unit) for message passing."""
    w0, w1 = int(wires[0]), int(wires[1])

    qml.RZ(bond_params[0], wires=w0)
    qml.RX(bond_params[1], wires=w0)
    qml.RZ(bond_params[0], wires=w1)
    qml.RX(bond_params[1], wires=w1)
    qml.CNOT(wires=[w0, w1])
    qml.RZ(rzz_param, wires=w1)
    qml.CNOT(wires=[w0, w1])
    qml.RX(bond_params[0], wires=w0)
    qml.RZ(bond_params[1], wires=w0)
    qml.RX(bond_params[0], wires=w1)
    qml.RZ(bond_params[1], wires=w1)

In [ ]:
class QuantumNet(nn.Module):
    """Quantum Neural Network Layer for Message Passing in Graphs."""
    def __init__(self, num_qubits, num_layers):
        super().__init__()
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.params = nn.Parameter(torch.randn(num_layers * 4))

        self.dev = qml.device('default.qubit', wires=num_qubits)

    def forward(self, x, edge_index):
        batch_size = x.shape[0]
        num_nodes = min(batch_size, self.num_qubits)
        x = x[:num_nodes]
        edge_index = edge_index[:, (edge_index[0] < num_nodes) & (edge_index[1] < num_nodes)]

        @qml.qnode(self.dev, interface='torch', diff_method="backprop")
        def circuit():
            for i in range(num_nodes):
                qml.RY(x[i][0], wires=i)
                qml.RZ(x[i][1], wires=i)

            for layer in range(self.num_layers):
                rzz_param = self.params[layer]
                for start, end in edge_index.T:
                    default_EDU(self.params[layer:layer+2], rzz_param, [int(start), int(end)])

            return [qml.expval(qml.PauliZ(i)) for i in range(num_nodes)]

        output = torch.tensor(circuit(), dtype=torch.float32)
        return output.unsqueeze(0).expand(batch_size, -1)



class QGCNConv(torch_geometric.nn.GCNConv):
    """Quantum Graph Convolution Layer (GCN + QuantumNet)."""
    def __init__(self, in_channels, out_channels, num_layers=1, **kwargs):
        super().__init__(in_channels, out_channels, **kwargs)
        self.quantum_layer = QuantumNet(in_channels, num_layers)

    def forward(self, x, edge_index):
        return self.quantum_layer(x, edge_index)

In [ ]:
class GraphGNNModel(nn.Module):
    def __init__(self, c_in, c_hidden, c_out, **kwargs):
        super().__init__()
        self.embed = nn.Linear(c_in, c_hidden)
        self.GNN = QGCNConv(c_hidden, c_hidden, num_layers=2, **kwargs)
        self.head = nn.Linear(c_hidden, c_out)

    def forward(self, x, edge_index, batch_idx):
        batch_size = x.shape[0]
        batch_idx = torch.clamp(batch_idx[:batch_size], min=0, max=batch_size - 1)

        x = self.embed(x)
        x = self.GNN(x, edge_index)

        batch_idx = batch_idx[:x.shape[0]]
        x = global_mean_pool(x, batch_idx)

        return self.head(x)


class GraphLevelGNN(pl.LightningModule):
    def __init__(self, c_in, c_hidden, c_out, **model_kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.model = GraphGNNModel(c_in=c_in, c_hidden=c_hidden, c_out=c_out, **model_kwargs)
        self.loss_module = nn.CrossEntropyLoss()

    def forward(self, batch):
        x, edge_index, batch_idx = batch.x, batch.edge_index, batch.batch
        x = self.model(x, edge_index, batch_idx)
        return x

    def training_step(self, batch, batch_idx):
        y_pred = self.forward(batch)
        loss = self.loss_module(y_pred, batch.y.view(-1))
        acc = (y_pred.argmax(dim=1) == batch.y.view(-1)).float().mean()
        self.log('train_loss', loss, prog_bar=True)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        y_pred = self.forward(batch)
        loss = self.loss_module(y_pred, batch.y.view(-1))
        acc = (y_pred.argmax(dim=1) == batch.y.view(-1)).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
def train_qgnn(num_epochs=50):
    trainer = pl.Trainer(accelerator="gpu" if torch.cuda.is_available() else "cpu", devices=1, max_epochs=num_epochs)
    model = GraphLevelGNN(c_in=4, c_hidden=6, c_out=2)
    trainer.fit(model, train_loader, val_loader)
    return model

best_model = train_qgnn()


Using device: cpu
Loading dataset...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name        | Type             | Params | Mode 
---------------------------------------------------------
0 | model       | GraphGNNModel    | 94     | train
1 | loss_module | CrossEntropyLoss | 0      | train
---------------------------------------------------------
94        Trainable params
0         Non-trainable params
94        Total params
0.000     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4448. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
